# Python для анализа данных

## Бот в Telegram

### Сбор статистики по Коронавирусу

*Ян Пиле, Яндекс.Маркет*

### Есть сайт с информацией о распространении вируса в мире
Я предлагаю достать оттуда информацию в виде Pandas dataframe а затем по запросу пользователя в телеграм выгружать ее

Импортируем requests и запишем адрес сайта со статистикой по COVID в переменную

In [2]:
import requests
url1 = 'https://www.worldometers.info/coronavirus/'

После чего через get-запрос достанем текст страницы со статистикой и распарсим ее с помощью BeautifulSoup

In [3]:
website_url = requests.get(url1).text
from bs4 import BeautifulSoup
soup = BeautifulSoup(website_url, 'lxml')
# soup

На странице две таблицы: одна отвечает за информацию на сегодняшний день (под номером 0), а вторая - за информацию на вчерашний день. Достанем обе таблицы (они лежат между тегами \<\table\>\)

In [22]:
table = soup.find_all('table')[0]
# table

Разобьем таблицу на строки (они лежат между тегами tr)

In [21]:
rows = table.find_all('tr')
# rows

Сначала достанем заголовки (между тегами th). Для примера достанем заголовок первой (нулевой) колонки.

In [7]:
rows[0].find_all('th')[0].get_text().strip()

u'Country,Other'

Теперь мы можем достать содержимое ячеек таблицы (между тегами td). Для примера достанем первую запись второго по счету(и первого по номеру) столбца

In [8]:
rows[1].find_all('td')[1].get_text().strip()

u'908,231'

Протестируем сборку на примере одной колонки, а потом пробежимся по их списку в цикле

In [9]:
col1 = []

col1.append(rows[0].find_all('th')[0].get_text().strip()) # отдельно добавляем заголовок

for row in rows[1:]: # начинаем со второго ряда таблицы, потому что 0 уже обработали выше
    r = row.find_all('td') # находим все теги td для строки таблицы
    col1.append(r[0].get_text().strip()) # сохраняем данные в наш список

print(col1)

[u'Country,Other', u'North America', u'Europe', u'Asia', u'South America', u'Oceania', u'Africa', u'', u'World', u'USA', u'Spain', u'Italy', u'France', u'Germany', u'UK', u'Turkey', u'Iran', u'Russia', u'Brazil', u'Belgium', u'Canada', u'Netherlands', u'Switzerland', u'Portugal', u'India', u'Peru', u'Ireland', u'Sweden', u'Austria', u'Israel', u'Saudi Arabia', u'Japan', u'Chile', u'Ecuador', u'S. Korea', u'Poland', u'Singapore', u'Pakistan', u'Romania', u'Mexico', u'UAE', u'Denmark', u'Indonesia', u'Belarus', u'Norway', u'Qatar', u'Serbia', u'Czechia', u'Philippines', u'Australia', u'Ukraine', u'Malaysia', u'Dominican Republic', u'Panama', u'Colombia', u'Finland', u'Bangladesh', u'Egypt', u'Luxembourg', u'South Africa', u'Morocco', u'Argentina', u'Algeria', u'Thailand', u'Moldova', u'Greece', u'Kuwait', u'Hungary', u'Kazakhstan', u'Bahrain', u'Croatia', u'Iceland', u'Uzbekistan', u'Iraq', u'Oman', u'Estonia', u'Azerbaijan', u'Armenia', u'New Zealand', u'Lithuania', u'Bosnia and Herzego

Чтобы еще раз проверить, запустим для другой колонки - TotalCases

In [10]:
col2 = []

col2.append(rows[0].find_all('th')[1].get_text().strip()) # отдельно добавляем заголовок

for row in rows[1:]: # начинаем со второго ряда таблицы, потому что 0 уже обработали выше
    r = row.find_all('td') # находим все теги td для строки таблицы
    col2.append(r[1].get_text().strip()) # сохраняем данные в наш список

print(col2)

[u'TotalCases', u'908,231', u'1,167,590', u'420,949', u'96,284', u'8,201', u'26,493', u'721', u'2,628,469', u'844,440', u'208,389', u'187,327', u'159,877', u'150,062', u'133,495', u'98,674', u'85,996', u'57,999', u'45,757', u'41,889', u'39,813', u'34,842', u'28,268', u'21,982', u'20,471', u'19,250', u'16,671', u'16,004', u'14,925', u'14,498', u'12,772', u'11,512', u'11,296', u'10,850', u'10,694', u'10,169', u'10,141', u'10,076', u'9,710', u'9,501', u'8,238', u'7,912', u'7,418', u'7,281', u'7,275', u'7,141', u'7,114', u'7,087', u'6,710', u'6,649', u'6,592', u'5,532', u'5,300', u'4,821', u'4,149', u'4,129', u'3,772', u'3,659', u'3,654', u'3,635', u'3,446', u'3,144', u'2,910', u'2,826', u'2,778', u'2,408', u'2,248', u'2,168', u'2,135', u'2,027', u'1,950', u'1,785', u'1,716', u'1,631', u'1,614', u'1,559', u'1,518', u'1,473', u'1,451', u'1,370', u'1,368', u'1,353', u'1,259', u'1,244', u'1,189', u'1,176', u'1,163', u'1,154', u'1,034', u'1,024', u'974', u'916', u'909', u'790', u'782', u'761',

А теперь пройдемся циклом по всем 12 колонкам, сгружая данные в список списков

In [11]:
field_list = []
for i in range(12):
    col = []
    col.append(rows[0].find_all('th')[i].get_text().strip()) # отдельно добавляем заголовок
    for row in rows[1:]: # начинаем со второго ряда таблицы, потому что 0 уже обработали выше
        r = row.find_all('td') # находим все теги td для строки таблицы
        col.append(r[i].get_text().strip()) # сохраняем данные в наш список
    field_list.append(col)

А вот заголовок второго столбца

In [12]:
field_list[1][0]

u'TotalCases'

Остается превратить эту структуру в DataFrsme. Для этого сначала превратим ее в словарь из списков, где ключами будут названия колонок.

In [26]:
d = dict()
for i in range(12):
    d[field_list[i][0]] = field_list[i][1:]
# d

А теперь сгружаем этот словарь в функцию pd.DataFrame и он превращается в искомый DataFrame

In [27]:
import pandas as pd 
df = pd.DataFrame(d)
df.head(10)

,ActiveCases,"Country,Other",Deaths/1M pop,NewCases,NewDeaths,"Serious,Critical",Tests/ 1M pop,TotalCases,TotalDeaths,TotalRecovered,TotalTests,Tot Cases/1M pop
0,"755,606",North America,,"+28,364","+2,211","15,269",,"908,231","50,647","101,978",,
1,"695,663",Europe,,"+27,539","+3,231","25,509",,"1,167,590","111,137","360,790",,
2,"206,948",Asia,,"+10,689",+298,"6,169",,"420,949","15,807","198,194",,
3,"51,397",South America,,"+5,024",+246,"9,499",,"96,284","4,558","40,329",,
4,"2,097",Oceania,,+11,+4,51,,"8,201",88,"6,016",,
5,"18,020",Africa,,"+1,082",+42,174,,"26,493","1,239","7,234",,
6,62,,,,,7,,721,15,644,,
7,"1,729,793",World,23.5,"+72,709","+6,032","56,678",,"2,628,469","183,491","715,185",,337
8,"713,454",USA,143,"+25,696","+1,909","14,014","13,009","844,440","47,227","83,759","4,305,949","2,551"
9,"100,757",Spain,464,"+4,211",+435,"7,705","19,896","208,389","21,717","85,915","930,230","4,457"


In [17]:
df.head(8)

,ActiveCases,"Country,Other",Deaths/1M pop,NewCases,NewDeaths,"Serious,Critical",Tests/ 1M pop,TotalCases,TotalDeaths,TotalRecovered,TotalTests,Tot Cases/1M pop
0,"755,606",North America,,"+28,364","+2,211","15,269",,"908,231","50,647","101,978",,
1,"695,663",Europe,,"+27,539","+3,231","25,509",,"1,167,590","111,137","360,790",,
2,"206,948",Asia,,"+10,689",+298,"6,169",,"420,949","15,807","198,194",,
3,"51,397",South America,,"+5,024",+246,"9,499",,"96,284","4,558","40,329",,
4,"2,097",Oceania,,+11,+4,51,,"8,201",88,"6,016",,
5,"18,020",Africa,,"+1,082",+42,174,,"26,493","1,239","7,234",,
6,62,,,,,7,,721,15,644,,
7,"1,729,793",World,23.5,"+72,709","+6,032","56,678",,"2,628,469","183,491","715,185",,337


Для удобства переименуем колонки с составными названиями.

In [131]:
df = df.rename(columns={'Country,Other': 'Country', 'Serious,Critical': 'SeriousCritical'})

А теперь сгрузим весь процесс, который мы проделали, в одну функцию. Ее аргумент tag = 0 - информация за сегодняшний день, a tag=1 - информация за вчерашний день.

In [4]:
def stat(tag=0):
    url = 'https://www.worldometers.info/coronavirus/'
    website_url = requests.get(url).text
    soup = BeautifulSoup(website_url, 'lxml')
    table = soup.find_all('table')[tag]
    rows = table.find_all('tr')
    field_list = []
    for i in range(12):
        col = []
        col.append(rows[0].find_all('th')[i].get_text().strip())  # отдельно добавляем заголовок
        for row in rows[1:]:  # начинаем со второго ряда таблицы, потому что 0 уже обработали выше
            r = row.find_all('td')  # находим все теги td для строки таблицы
            col.append(r[i].get_text().strip())  # сохраняем данные в наш список
        field_list.append(col)
    d = dict()
    for i in range(12):
        d[field_list[i][0]] = field_list[i][1:]

    df = pd.DataFrame(d)
    df = df.rename(columns={'Country,Other': 'Country', 'Serious,Critical': 'SeriousCritical'})
    return df

В первых 8 строках таблицы в колонке Country лежат регионы + Мир. Одна строка пустая. Уберем ее с помощью спискового включения.

In [170]:
print(*list(i+'\n' for i in list(stat()['Country'][:8]) if i!=''))

North America
 Europe
 Asia
 South America
 Oceania
 Africa
 World



А с 8 по 220 строки лежат названия стран. Выведем ее с помощью спискового включения по 6 стран в линию (для этого вставим символ переноса строки)

In [173]:
x = stat()['Country'][8:220]
import re
# print(x)
print(', '.join([e+'\n' if i%6 == 5 else e for i,e in enumerate(x)]).replace('\n,',',\n'))


USA, Spain, Italy, France, Germany, UK,
 Iran, Turkey, Belgium, Netherlands, Canada, Switzerland,
 Brazil, Russia, Portugal, Austria, Israel, Ireland,
 Sweden, India, S. Korea, Peru, Chile, Japan,
 Ecuador, Poland, Romania, Norway, Denmark, Australia,
 Czechia, Pakistan, Saudi Arabia, Philippines, Mexico, Malaysia,
 UAE, Indonesia, Serbia, Panama, Qatar, Ukraine,
 Luxembourg, Dominican Republic, Belarus, Singapore, Finland, Colombia,
 Thailand, South Africa, Egypt, Argentina, Greece, Algeria,
 Moldova, Morocco, Iceland, Croatia, Bahrain, Hungary,
 Iraq, Estonia, New Zealand, Kuwait, Kazakhstan, Slovenia,
 Azerbaijan, Uzbekistan, Bosnia and Herzegovina, Lithuania, Armenia, Hong Kong,
 Bangladesh, North Macedonia, Cameroon, Slovakia, Oman, Cuba,
 Tunisia, Afghanistan, Bulgaria, Diamond Princess, Cyprus, Latvia,
 Andorra, Lebanon, Ghana, Ivory Coast, Costa Rica, Niger,
 Burkina Faso, Uruguay, Albania, Channel Islands, Kyrgyzstan, Honduras,
 Jordan, Taiwan, Malta, Réunion, San Marino, Djib

Для вывода результатов в текстовом формате воспользуемся библиотекой tabulate (в таком формате мы будем отправлять результаты в telegram'е)

In [139]:
from tabulate import tabulate

Например общее количество случаев заболевания коронавирусом в Великобритании вывести можно так:

In [140]:
x = df[['Country','TotalCases']][df['Country']=='UK'].reset_index(drop=True)
print(tabulate(x, headers = x.columns, tablefmt="grid"))

  Country TotalCases
0      UK     88,621
+----+-----------+--------------+
|    | Country   | TotalCases   |
+====+===========+==============+
|  0 | UK        | 88,621       |
+----+-----------+--------------+
